In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Load the data
df = pd.read_csv('nyc_housing_processed.csv')

# Display basic information
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head())
print("\nData types:")
print(df.dtypes)
print("\nMissing values:")
print(df.isnull().sum())

In [0]:
# Data preprocessing and feature engineering
def preprocess_data(df):
    """
    Preprocess the NYC housing data
    """
    # Create a copy
    df_processed = df.copy()
    
    # Remove outliers in sale_price (keeping reasonable range)
    q1 = df_processed['sale_price'].quantile(0.01)
    q99 = df_processed['sale_price'].quantile(0.99)
    df_processed = df_processed[(df_processed['sale_price'] >= q1) & 
                                  (df_processed['sale_price'] <= q99)]
    
    # Feature Engineering
    # 1. Total area (if individual areas are 0)
    df_processed['total_area'] = df_processed['bldgarea'].fillna(0) + \
                                  df_processed['resarea'].fillna(0) + \
                                  df_processed['comarea'].fillna(0)
    
    # 2. Area per unit
    df_processed['area_per_unit'] = df_processed['bldgarea'] / (df_processed['unitstotal'] + 1)
    
    # 3. Residential ratio
    df_processed['residential_ratio'] = df_processed['resarea'] / (df_processed['bldgarea'] + 1)
    
    # 4. Commercial ratio
    df_processed['commercial_ratio'] = df_processed['comarea'] / (df_processed['bldgarea'] + 1)
    
    # 5. Floor area ratio
    df_processed['floor_area_ratio'] = df_processed['bldgarea'] / (df_processed['lotarea'] + 1)
    
    # 6. Price per floor
    df_processed['price_per_floor'] = df_processed['sale_price'] / (df_processed['numfloors'] + 1)
    
    # 7. Location features - proximity to Manhattan (assuming Manhattan is at center)
    manhattan_lat, manhattan_lon = 40.7831, -73.9712
    df_processed['distance_to_manhattan'] = np.sqrt(
        (df_processed['latitude'] - manhattan_lat)**2 + 
        (df_processed['longitude'] - manhattan_lon)**2
    )
    
    # Select features for modeling
    feature_columns = [
        # Numerical features
        'yearbuilt', 'lotarea', 'bldgarea', 'resarea', 'comarea',
        'unitsres', 'unitstotal', 'numfloors', 'latitude', 'longitude',
        'building_age', 'price_per_sqft', 
        # Engineered features
        'total_area', 'area_per_unit', 'residential_ratio', 'commercial_ratio',
        'floor_area_ratio', 'distance_to_manhattan',
        # Categorical features
        'borough_name', 'building_type', 'building_category', 'use_type',
        'price_tier', 'size_category', 'age_category', 'zip_code'
    ]
    
    # Target variable
    target = 'sale_price'
    
    # Remove rows with missing target
    df_processed = df_processed.dropna(subset=[target])
    
    # Select only available columns
    available_features = [col for col in feature_columns if col in df_processed.columns]
    
    return df_processed, available_features, target

# Preprocess the data
df_processed, feature_columns, target = preprocess_data(df)
print(f"\nProcessed dataset shape: {df_processed.shape}")
print(f"Number of features: {len(feature_columns)}")

In [0]:
# Exploratory Data Analysis
def perform_eda(df, target):
    """
    Perform basic EDA on the dataset
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Sale price distribution
    axes[0, 0].hist(df[target], bins=50, edgecolor='black')
    axes[0, 0].set_title('Sale Price Distribution')
    axes[0, 0].set_xlabel('Sale Price')
    axes[0, 0].set_ylabel('Frequency')
    
    # Log-transformed sale price
    axes[0, 1].hist(np.log1p(df[target]), bins=50, edgecolor='black')
    axes[0, 1].set_title('Log Sale Price Distribution')
    axes[0, 1].set_xlabel('Log(Sale Price)')
    
    # Sale price by borough
    if 'borough_name' in df.columns:
        borough_prices = df.groupby('borough_name')[target].median().sort_values()
        axes[0, 2].bar(range(len(borough_prices)), borough_prices.values)
        axes[0, 2].set_xticks(range(len(borough_prices)))
        axes[0, 2].set_xticklabels(borough_prices.index, rotation=45)
        axes[0, 2].set_title('Median Price by Borough')
    
    # Building age vs price
    if 'building_age' in df.columns:
        axes[1, 0].scatter(df['building_age'], df[target], alpha=0.5)
        axes[1, 0].set_xlabel('Building Age')
        axes[1, 0].set_ylabel('Sale Price')
        axes[1, 0].set_title('Building Age vs Sale Price')
    
    # Building area vs price
    if 'bldgarea' in df.columns:
        axes[1, 1].scatter(df['bldgarea'], df[target], alpha=0.5)
        axes[1, 1].set_xlabel('Building Area')
        axes[1, 1].set_ylabel('Sale Price')
        axes[1, 1].set_title('Building Area vs Sale Price')
    
    # Price tier distribution
    if 'price_tier' in df.columns:
        price_tier_counts = df['price_tier'].value_counts()
        axes[1, 2].bar(range(len(price_tier_counts)), price_tier_counts.values)
        axes[1, 2].set_xticks(range(len(price_tier_counts)))
        axes[1, 2].set_xticklabels(price_tier_counts.index, rotation=45)
        axes[1, 2].set_title('Price Tier Distribution')
    
    plt.tight_layout()
    plt.show()
    
    # Correlation matrix for numerical features
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    if len(numerical_cols) > 1:
        plt.figure(figsize=(12, 8))
        correlation_matrix = df[numerical_cols].corr()
        sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0)
        plt.title('Feature Correlation Matrix')
        plt.show()

# Perform EDA
perform_eda(df_processed, target)

In [0]:
# Prepare features and target
def prepare_features(df, feature_columns, target):
    """
    Prepare features for machine learning
    """
    # Separate numerical and categorical columns
    numerical_features = []
    categorical_features = []
    
    for col in feature_columns:
        if col in df.columns:
            if df[col].dtype in ['object', 'category']:
                categorical_features.append(col)
            else:
                numerical_features.append(col)
    
    # Create feature matrix
    X = pd.DataFrame()
    
    # Add numerical features
    for col in numerical_features:
        X[col] = df[col].fillna(df[col].median())
    
    # Encode categorical features
    for col in categorical_features:
        if df[col].nunique() < 20:  # One-hot encode if few categories
            dummies = pd.get_dummies(df[col], prefix=col, drop_first=True)
            X = pd.concat([X, dummies], axis=1)
        else:  # Label encode if many categories
            le = LabelEncoder()
            X[col] = le.fit_transform(df[col].fillna('missing').astype(str))
    
    # Get target
    y = df[target].values
    
    # Remove any infinite values
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.fillna(X.median())
    
    return X, y

# Prepare features
X, y = prepare_features(df_processed, feature_columns, target)
print(f"\nFeature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Log transform the target for better distribution
y_log = np.log1p(y)

In [0]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_log, test_size=0.2, random_state=42
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set size: {X_train_scaled.shape}")
print(f"Test set size: {X_test_scaled.shape}")

In [0]:
# Model Training and Evaluation
def train_models(X_train, y_train, X_test, y_test):
    """
    Train multiple models and compare performance
    """
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(alpha=1.0),
        'Lasso Regression': Lasso(alpha=0.01),
        'Random Forest': RandomForestRegressor(
            n_estimators=100,
            max_depth=20,
            min_samples_split=5,
            random_state=42,
            n_jobs=-1
        ),
        'Gradient Boosting': GradientBoostingRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            random_state=42
        )
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\nTraining {name}...")
        
        # Train model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        
        # Convert back from log scale
        y_pred_train_exp = np.expm1(y_pred_train)
        y_pred_test_exp = np.expm1(y_pred_test)
        y_train_exp = np.expm1(y_train)
        y_test_exp = np.expm1(y_test)
        
        # Calculate metrics
        train_rmse = np.sqrt(mean_squared_error(y_train_exp, y_pred_train_exp))
        test_rmse = np.sqrt(mean_squared_error(y_test_exp, y_pred_test_exp))
        train_mae = mean_absolute_error(y_train_exp, y_pred_train_exp)
        test_mae = mean_absolute_error(y_test_exp, y_pred_test_exp)
        train_r2 = r2_score(y_train_exp, y_pred_train_exp)
        test_r2 = r2_score(y_test_exp, y_pred_test_exp)
        
        results[name] = {
            'model': model,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'train_mae': train_mae,
            'test_mae': test_mae,
            'train_r2': train_r2,
            'test_r2': test_r2,
            'y_pred_test': y_pred_test_exp
        }
        
        print(f"  Train RMSE: ${train_rmse:,.0f}")
        print(f"  Test RMSE: ${test_rmse:,.0f}")
        print(f"  Train MAE: ${train_mae:,.0f}")
        print(f"  Test MAE: ${test_mae:,.0f}")
        print(f"  Train R²: {train_r2:.4f}")
        print(f"  Test R²: {test_r2:.4f}")
    
    return results

# Train models
results = train_models(X_train_scaled, y_train, X_test_scaled, y_test)

In [0]:
# Visualize model performance
def plot_model_comparison(results):
    """
    Create visualizations comparing model performance
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Extract metrics
    model_names = list(results.keys())
    test_rmse = [results[m]['test_rmse'] for m in model_names]
    test_mae = [results[m]['test_mae'] for m in model_names]
    test_r2 = [results[m]['test_r2'] for m in model_names]
    
    # RMSE comparison
    axes[0, 0].bar(model_names, test_rmse)
    axes[0, 0].set_title('Test RMSE by Model')
    axes[0, 0].set_ylabel('RMSE ($)')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # MAE comparison
    axes[0, 1].bar(model_names, test_mae)
    axes[0, 1].set_title('Test MAE by Model')
    axes[0, 1].set_ylabel('MAE ($)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # R² comparison
    axes[1, 0].bar(model_names, test_r2)
    axes[1, 0].set_title('Test R² by Model')
    axes[1, 0].set_ylabel('R² Score')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Best model predictions vs actual
    best_model_name = max(results.keys(), key=lambda k: results[k]['test_r2'])
    y_pred_best = results[best_model_name]['y_pred_test']
    y_test_exp = np.expm1(y_test)
    
    axes[1, 1].scatter(y_test_exp, y_pred_best, alpha=0.5)
    axes[1, 1].plot([y_test_exp.min(), y_test_exp.max()], 
                    [y_test_exp.min(), y_test_exp.max()], 
                    'r--', lw=2)
    axes[1, 1].set_xlabel('Actual Price ($)')
    axes[1, 1].set_ylabel('Predicted Price ($)')
    axes[1, 1].set_title(f'Best Model ({best_model_name}) - Predictions vs Actual')
    
    plt.tight_layout()
    plt.show()
    
    return best_model_name

# Plot comparisons
best_model_name = plot_model_comparison(results)
print(f"\nBest performing model: {best_model_name}")

In [0]:
# Feature importance analysis for tree-based models
def analyze_feature_importance(model, feature_names, model_name):
    """
    Analyze and plot feature importance for tree-based models
    """
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1][:20]  # Top 20 features
        
        plt.figure(figsize=(12, 6))
        plt.title(f'Top 20 Feature Importances - {model_name}')
        plt.bar(range(len(indices)), importances[indices])
        plt.xticks(range(len(indices)), [feature_names[i] for i in indices], rotation=90)
        plt.xlabel('Features')
        plt.ylabel('Importance')
        plt.tight_layout()
        plt.show()
        
        # Print top 10 features
        print(f"\nTop 10 Most Important Features for {model_name}:")
        for i in range(min(10, len(indices))):
            print(f"{i+1}. {feature_names[indices[i]]}: {importances[indices[i]]:.4f}")

# Analyze feature importance for Random Forest
if 'Random Forest' in results:
    analyze_feature_importance(
        results['Random Forest']['model'],
        X.columns,
        'Random Forest'
    )

In [0]:
# Hyperparameter tuning for the best model
def tune_best_model(X_train, y_train, X_test, y_test, model_type='random_forest'):
    """
    Perform hyperparameter tuning for the best model
    """
    print("Performing hyperparameter tuning...")
    
    if model_type == 'random_forest':
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
        base_model = RandomForestRegressor(random_state=42, n_jobs=-1)
    else:  # gradient_boosting
        param_grid = {
            'n_estimators': [100, 200],
            'learning_rate': [0.05, 0.1, 0.15],
            'max_depth': [3, 5, 7],
            'min_samples_split': [2, 5, 10]
        }
        base_model = GradientBoostingRegressor(random_state=42)
    
    # Grid search with cross-validation
    grid_search = GridSearchCV(
        base_model,
        param_grid,
        cv=3,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1
    )
    
    grid_search.fit(X_train, y_train)
    
    # Best model
    best_model = grid_search.best_estimator_
    print(f"\nBest parameters: {grid_search.best_params_}")
    
    # Evaluate best model
    y_pred_test = best_model.predict(X_test)
    y_pred_test_exp = np.expm1(y_pred_test)
    y_test_exp = np.expm1(y_test)
    
    test_rmse = np.sqrt(mean_squared_error(y_test_exp, y_pred_test_exp))
    test_mae = mean_absolute_error(y_test_exp, y_pred_test_exp)
    test_r2 = r2_score(y_test_exp, y_pred_test_exp)
    
    print(f"\nTuned Model Performance:")
    print(f"  Test RMSE: ${test_rmse:,.0f}")
    print(f"  Test MAE: ${test_mae:,.0f}")
    print(f"  Test R²: {test_r2:.4f}")
    
    return best_model

# Tune the best model (uncomment to run - takes time)
# tuned_model = tune_best_model(X_train_scaled, y_train, X_test_scaled, y_test)

In [0]:
# Create prediction function for new data
def predict_price(model, scaler, new_data, feature_columns):
    """
    Predict price for new property
    """
    # Prepare input data
    new_df = pd.DataFrame([new_data])
    X_new, _ = prepare_features(new_df, feature_columns, 'sale_price')
    
    # Scale features
    X_new_scaled = scaler.transform(X_new)
    
    # Make prediction
    price_log = model.predict(X_new_scaled)[0]
    price = np.expm1(price_log)
    
    return price

# Example prediction (you would need to provide actual values)
"""
example_property = {
    'yearbuilt': 2000,
    'lotarea': 2500,
    'bldgarea': 3000,
    'resarea': 2800,
    'comarea': 200,
    'unitsres': 1,
    'unitstotal': 1,
    'numfloors': 2,
    'latitude': 40.7,
    'longitude': -73.9,
    'building_age': 23,
    'borough_name': 'Manhattan',
    'building_type': 'residential',
    # ... other features
}

predicted_price = predict_price(
    results[best_model_name]['model'],
    scaler,
    example_property,
    feature_columns
)
print(f"Predicted price: ${predicted_price:,.0f}")
"""

# Summary
print("\n" + "="*50)
print("MODEL TRAINING COMPLETE")
print("="*50)
print(f"\nBest Model: {best_model_name}")
print(f"Test R² Score: {results[best_model_name]['test_r2']:.4f}")
print(f"Test RMSE: ${results[best_model_name]['test_rmse']:,.0f}")
print(f"Test MAE: ${results[best_model_name]['test_mae']:,.0f}")